In [1]:
import os, random, pickle
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, log_loss
from nilearn import datasets, image, plotting
import matplotlib.pyplot as plt

/Users/ehbeam/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1. Evaluate top models in each class on the test set

_Models are logistic regression classifiers predicting whether or not an activation coordinate was reported within each structure of the Harvard-Oxford atlas based on psychological features in the article full text_

_Top models in each class were identified by a grid search over 200 random combinations of hyperparameters, including penalty, inverse regularization strength (c), intercept, and maximum number of iterations_ 

### Functions for classifier training and prediction

In [2]:
def train(classifier, activation_atlas, features, verbose=False):
    fits, count = {}, 0
    for structure in activation_atlas.columns:
        if verbose:
            clear_output()
            print("Fitting classifier {}/{}: {}".format(count+1, len(activation_atlas.columns), structure))
        classifier.fit(features, activation_atlas[structure])
        fits[structure] = pickle.dumps(classifier)
        count += 1
    if verbose:
        clear_output()
        print("Trained classifiers for {} brain structures".format(count))
    return fits

In [3]:
def mind2brain(fits, activation_atlas, features):
    predictions = {}
    for structure in activation_atlas.columns:
        fit = pickle.loads(fits[structure])
        predictions[structure] = fit.predict(features)
    return predictions

### Split studies into training and test sets

In [4]:
activation_atlas = pd.read_csv("data/activations_prob0.csv", index_col=0, header=0)

In [5]:
random.seed(42)
train_prop, val_prop = 0.7, 0.2
ids = sorted(list(activation_atlas.index))
random.shuffle(ids)
train_ids = ids[:int(train_prop*len(ids))]
test_ids = ids[int((train_prop+val_prop)*len(ids)):]

### Load hyperparameters of the top models

In [6]:
top_models = pd.read_csv("data/top_models_180613.csv", index_col=2, header=0)

### Fit each top model on the test set

_The test set includes 1,094 articles withheld from training and hyperparameter optimization_

In [7]:
for model in top_models.index:
    clear_output()
    print("Fitting: {}".format(model))
    activation_atlas = pd.read_csv("data/activations_prob{}.csv".format(top_models.loc[model,"COORD_PROB_THRES"]), index_col=0, header=0)
    features = pd.read_csv("features/{}.{}".format("_".join(model.split("_")[:-1]), top_models.loc[model,"EXTENSION"]), index_col=0, header=0)
    classifier = LogisticRegression(penalty=top_models.loc[model,"PENALTY"],
                                C=top_models.loc[model,"C"], 
                                fit_intercept=top_models.loc[model,"INTERCEPT"],
                                max_iter=top_models.loc[model,"MAX_ITER"])
    fits = train(classifier, activation_atlas.loc[train_ids], features.loc[train_ids])
    pred = pd.DataFrame(mind2brain(fits, activation_atlas.loc[test_ids], features.loc[test_ids]))
    top_models.at[model,"TEST_MACRO_PRECISION"] = precision_score(activation_atlas.loc[test_ids], pred, average="macro")
    top_models.at[model,"TEST_MICRO_PRECISION"] = precision_score(activation_atlas.loc[test_ids], pred, average="micro")
    top_models.at[model,"TEST_MACRO_RECALL"] = recall_score(activation_atlas.loc[test_ids], pred, average="macro")
    top_models.at[model,"TEST_MICRO_RECALL"] = recall_score(activation_atlas.loc[test_ids], pred, average="micro")
    top_models.at[model,"TEST_MACRO_F1"] = f1_score(activation_atlas.loc[test_ids], pred, average="macro")
    top_models.at[model,"TEST_MICRO_F1"] = f1_score(activation_atlas.loc[test_ids], pred, average="micro")
    top_models.at[model,"TEST_LOGLOSS"] = log_loss(activation_atlas.loc[test_ids], pred)
clear_output()
print("Fitting on test set complete!".format(model))

Fitting on test set complete!


In [8]:
top_models["TEST_MICRO_F1"].sort_values(ascending=False)

PARAMS
bags_cons_seeds_syns_0           0.543132
tokens_ontol_0                   0.539403
llda_cons_seeds_79_0             0.538662
llda_kmeans_seeds_syns_81_0      0.535224
bags_kmeans_seeds_syns_0         0.534795
tokens_seeds_syns_0              0.532715
llda_kmeans_seeds_67_0           0.532650
llda_cons_seeds_syns_40_0        0.532439
tokens_seeds_0                   0.528953
bags_cons_seeds_0                0.517346
bags_kmeans_seeds_0              0.517297
lda_2_0                          0.491537
tokens_ontol_25                  0.323560
tokens_seeds_syns_25             0.297761
tokens_seeds_25                  0.279747
bags_cons_seeds_syns_25          0.266699
bags_kmeans_seeds_syns_25        0.262401
llda_kmeans_seeds_syns_165_25    0.248126
bags_kmeans_seeds_25             0.242219
llda_cons_seeds_syns_123_25      0.240939
bags_cons_seeds_25               0.239780
llda_cons_seeds_123_25           0.238000
llda_kmeans_seeds_122_25         0.230934
tokens_ontol_50            

In [9]:
top_models.to_csv("data/test_performance_180613.csv")

In [10]:
top_models

,INDEX,MODEL,ORGANIZATION,LABELS,COORD_PROB_THRES,ALPHA,BETA,TRAIN_PERPLEXITY,VAL_PERPLEXITY,PENALTY,...,VAL_MICRO_F1,VAL_LOGLOSS,EXTENSION,TEST_MACRO_PRECISION,TEST_MICRO_PRECISION,TEST_MACRO_RECALL,TEST_MICRO_RECALL,TEST_MACRO_F1,TEST_MICRO_F1,TEST_LOGLOSS
PARAMS,,,,,,,,,,,,,,,,,,,,,
llda_cons_seeds_79_0,79,L-LDA,cons,seeds,0,9.713033,5.241644,65.357377,273.043966,l2,...,0.479286,472.102977,csv,0.495769,0.601134,0.354299,0.487953,0.349427,0.538662,428.358760
llda_cons_seeds_123_25,123,L-LDA,cons,seeds,25,7.383734,2.176274,58.942847,304.627694,l1,...,0.181899,324.379836,csv,0.295700,0.558175,0.078098,0.151245,0.102100,0.238000,308.169034
llda_cons_seeds_59_50,59,L-LDA,cons,seeds,50,6.512318,5.469003,61.512413,286.846788,l1,...,0.074591,96.788293,csv,0.122672,0.529412,0.018908,0.055857,0.028456,0.101053,93.101524
llda_cons_seeds_syns_40_0,40,L-LDA,cons,seeds_syns,0,5.542264,4.709795,58.483330,276.641236,l2,...,0.480201,475.305673,csv,0.504707,0.604669,0.339349,0.475624,0.328455,0.532439,436.868068
llda_cons_seeds_syns_123_25,123,L-LDA,cons,seeds_syns,25,7.383734,2.176274,57.598452,284.062100,l1,...,0.180761,315.800376,csv,0.316011,0.556115,0.079933,0.153783,0.104233,0.240939,308.152930
llda_cons_seeds_syns_88_50,88,L-LDA,cons,seeds_syns,50,3.413339,1.183433,51.394144,332.221050,l1,...,0.073147,97.823205,csv,0.123290,0.526971,0.019707,0.059116,0.029160,0.106306,101.965767
llda_kmeans_seeds_67_0,67,L-LDA,kmeans,seeds,0,8.381454,4.281660,62.580088,304.389066,l1,...,0.480852,471.396038,csv,0.491414,0.597989,0.348112,0.480182,0.343487,0.532650,433.830815
llda_kmeans_seeds_122_25,122,L-LDA,kmeans,seeds,25,9.104016,1.620958,60.083182,335.334900,l2,...,0.178241,337.337610,csv,0.259696,0.560429,0.072902,0.145431,0.093627,0.230934,313.311896
llda_kmeans_seeds_174_50,174,L-LDA,kmeans,seeds,50,3.449728,7.755293,62.165816,310.955300,l1,...,0.069804,91.250319,csv,0.135668,0.557480,0.017513,0.054926,0.026162,0.100000,92.454839


### Refit the classifier with the winning model

In [11]:
top_model = top_models["TEST_MICRO_F1"].sort_values(ascending=False).index[0]
top_model

'bags_cons_seeds_syns_0'

In [12]:
activation_atlas = pd.read_csv(
    "data/activations_prob{}.csv".format(top_models.loc[top_model,"COORD_PROB_THRES"]), 
    index_col=0, header=0)
features = pd.read_csv(
    "features/{}.csv".format("_".join(top_model.split("_")[:-1])), 
    index_col=0, header=0)

In [13]:
classifier = LogisticRegression(penalty=top_models.loc[top_model,"PENALTY"],
                                C=top_models.loc[top_model,"C"], 
                                fit_intercept=top_models.loc[top_model,"INTERCEPT"],
                                max_iter=top_models.loc[top_model,"MAX_ITER"])

In [14]:
fits = train(classifier, activation_atlas.loc[train_ids], features.loc[train_ids], verbose=True)

Trained classifiers for 56 brain structures


In [15]:
pred = pd.DataFrame(mind2brain(fits, activation_atlas.loc[test_ids], features.loc[test_ids]))

## 2. Visualize model coefficients on the brain and by structure

### Harvard-Oxford cortical and subcortical atlases

In [16]:
cor = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-1mm').maps
sub = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-1mm').maps

_Excuse the mess, but we need to make cortical and subcortical labels compatible_

In [17]:
sub_relab_map_r2l = {1:0, 2:0, 3:0, 12:0, 13:0, 14:0, 15:4, 16:5, 17:6, 18:7, 19:9, 20:10, 21:11}
sub_relab_map = {4:1, 5:2, 6:3, 7:4, 9:5, 10:6, 11:7}
sub_relab = image.load_img(sub).get_data()
for old, new in sub_relab_map_r2l.items():
    sub_relab[sub_relab == old] = new
for old, new in sub_relab_map.items():
    sub_relab[sub_relab == old] = new
sub_relab = sub_relab + 48
sub_relab[sub_relab == 48] = 0
cor_relab = image.load_img(cor).get_data()
relab = np.add(sub_relab, cor_relab)
relab[relab > 57] = 0
img = image.new_img_like(cor, relab)

### Matrix of model coefficients sorted by atlas labels

In [18]:
labels = pd.read_csv("data/harvard-oxford.csv", index_col=2, header=0)

In [19]:
coef = pd.DataFrame(index=activation_atlas.columns, columns=features.columns)
for structure in activation_atlas.columns:
    coef.loc[structure] = pickle.loads(fits[structure]).coef_

In [20]:
coef["ATLAS"] = labels.loc[coef.index, "INDEX"]
coef = coef.sort_values(["ATLAS"])

KeyError: "None of [Index(['accumbens', 'amygdala', 'angular_gyrus', 'brainstem', 'caudate',\n       'central_opercular_cortex', 'cingulate_gyrus_anterior_division',\n       'cingulate_gyrus_posterior_division', 'cuneal_cortex',\n       'frontal_medial_cortex', 'frontal_operculum_cortex',\n       'frontal_orbital_cortex', 'frontal_pole', 'heschls_gyrus',\n       'hippocampus', 'inferior_frontal_gyrus_pars_opercularis',\n       'inferior_frontal_gyrus_pars_triangularis',\n       'inferior_temporal_gyrus_anterior_division',\n       'inferior_temporal_gyrus_posterior_division',\n       'inferior_temporal_gyrus_temporooccipital_part', 'insular_cortex',\n       'intracalcarine_cortex', 'lateral_occipital_cortex_inferior_division',\n       'lateral_occipital_cortex_superior_division', 'lingual_gyrus',\n       'middle_frontal_gyrus', 'middle_temporal_gyrus_anterior_division',\n       'middle_temporal_gyrus_posterior_division',\n       'middle_temporal_gyrus_temporooccipital_part',\n       'oc

### Brain maps of coefficients for model features

In [ ]:
plt.rcParams["font.family"] = "Arial"
for feature in coef.columns[:-1]:
    feature_map = image.copy_img(img).get_data()
    for i, value in enumerate(coef[feature]):
        feature_map[feature_map == i+1] = value
    feature_map = image.new_img_like(img, feature_map)
    display = plotting.plot_stat_map(feature_map,
                                             cmap="RdBu_r",
                                             threshold=0.01,
                                             alpha=0.7,
                                             cut_coords=(4,-16,9),
                                             title=feature.replace("_", " ").title(),
                                             annotate=False,
                                             draw_cross=False)
    #display.savefig("figures/brainvis_{}.png".format(feature), dpi=250) 
    plotting.show()
    display.close()

In [ ]:
len(test_ids)

### Bar charts of the most predictive features for select structures

In [ ]:
labels["PREPROCESSED"]=labels.index

In [ ]:
plt.rcParams["font.family"] = "Arial"
for structure in ["accumbens", "amygdala", "brainstem", "cingulate_gyrus_anterior_division", 
                  "frontal_medial_cortex", "inferior_temporal_gyrus_temporooccipital_part", 
                  "insular_cortex", "lateral_occipital_cortex_inferior_division", "middle_frontal_gyrus", 
                  "middle_temporal_gyrus_posterior_division", "subcallosal_cortex", "superior_frontal_gyrus", 
                  "superior_parietal_lobule", "superior_temporal_gyrus_posterior_division", 
                  "temporal_fusiform_cortex_posterior_division"]:
    top_features = coef.loc[structure].sort_values(ascending=False)[1:11]
    fig, ax = plt.subplots()
    ax.bar(range(10), top_features, align="center", alpha=0.85, color="#CE7D69")
    ax.set_xticks(range(10))
    ax.set_xticklabels(top_features.index, rotation=70, ha="right")
    ax.set_ylabel("Coefficient weight")
    ax.set_title(labels.loc[structure,"PRESENTABLE"])
    plt.tight_layout()
    fig.savefig("figures/topfeat_{}.png".format(structure), dpi=250) 
    plt.show()

## 3. Apply the model to a randomly selected study not seen in training

_Let's begin by picking out a random study from the test set_

In [ ]:
random.seed(7)
PMID = random.sample(test_ids, 1)

### Model input and output

_Next, let's check out the text-based features for this article_

In [ ]:
pd.set_option("display.max_columns", len(features.columns))
features.loc[PMID]

_Now let's compare predicted and actual brain activations_

In [ ]:
pred = mind2brain(fits, activation_atlas.loc[PMID], features.loc[PMID])
pd.set_option("display.max_columns", len(activation_atlas.columns))
pred_vs_true = pd.DataFrame(pred).append(activation_atlas.loc[PMID])
pred_vs_true.index = ["PREDICTED", "TRUE"]
pred_vs_true

In [ ]:
print("\nPerformance on a randomly selected article (PMID={})\n".format(PMID[0]))
print("PRECISION:\t {0:.3f}".format(precision_score(pred_vs_true.loc["TRUE"], pred_vs_true.loc["PREDICTED"])))
print("RECALL:\t\t {0:.3f}".format(recall_score(pred_vs_true.loc["TRUE"], pred_vs_true.loc["PREDICTED"])))
print("F1:\t\t {0:.3f}".format(f1_score(pred_vs_true.loc["TRUE"], pred_vs_true.loc["PREDICTED"])))

### Visualization of model performance

_Rearrange the performance data to match the atlas labels_

In [ ]:
performance = pred_vs_true.transpose()
performance["ATLAS"] = labels.loc[performance.index, "INDEX"]
performance = performance.sort_values(["ATLAS"])

_Relabel as 100 for TP, 200 for FP, and -100 for FN_

In [ ]:
plt.rcParams["font.family"] = "Arial"
performance_map = image.copy_img(img).get_data()
for i, structure in enumerate(performance.index):
    t, p = performance.loc[structure,"TRUE"], performance.loc[structure,"PREDICTED"]
    if t == 1 and p == 1:
        performance_map[performance_map == i+1] = 100
    if t == 0 and p == 1:
        performance_map[performance_map == i+1] = 200
    if t == 1 and p == 0:
        performance_map[performance_map == i+1] = -100
performance_map = image.new_img_like(img, performance_map)

_Display true positives (purple), false positives (yellow), and false negatives (red)_ 

In [ ]:
display = plotting.plot_stat_map(performance_map,
                                         cmap="Set3",
                                         threshold=90,
                                         alpha=0.7,
                                         cut_coords=(4,-16,9),
                                         title="PMID={}".format(PMID[0]),
                                         annotate=False,
                                         draw_cross=False,
                                         colorbar=False)
plotting.show()
display.close()
print("TP=Purple, FP=Yellow, FN=Red")